In [1]:
import pandas as pd
import numpy as np
import json
from string_grouper import match_strings
from joblib import Parallel, delayed
import os
import swifter

In [2]:
os.chdir("C:\\Users\\fjying\\Desktop\\RA_Columbia\\patentparsing\\identifycompanypermno\\intermediate_output")

In [3]:
crosswalks = pd.read_csv("permno_processedfirstassignorname_crosswalk.csv")

In [4]:
crosswalks = crosswalks.rename(columns = {'name_0': 'company'})

In [5]:
companynames = pd.read_csv("patenttransfer_companynames_standardized.csv")

In [6]:
def remove_end_words(x, endwords):
    words = x.split()
    if words[-1] not in endwords or len(words) == 1:
        return x
    else:
        return ' '.join(words[:-1])
    
def remove_words(x, endwords):
    x = x.split()
    resultwords  = [word for word in x if word not in endwords]
    result = ' '.join(resultwords)
    return result

In [7]:
def find_overlap_words(df):    
    left_words = df['left_side'].split()
    right_words = df['right_side'].split()
    
    if len(left_words)== 1 & len(right_words) == 1:
        if df['left_side'] in df['right_side'] or df['right_side'] in df['left_side']:
            return True
        
    intersects = len(set(left_words) & set(right_words))
    
    if (len(left_words) > len(right_words)) & (intersects/len(left_words) > 0.5):
        return True
    if (len(left_words) <= len(right_words)) & (intersects/len(right_words)> 0.5):
        return True    
    return False

In [8]:
def single_words_name(df):
    left_words = df['left_side'].split()
    right_words = df['right_side'].split()

    if  len(left_words)== 1 or len(right_words) == 1:
        return True
    else:
        return False

In [10]:
def build_companynames_permno_crosswalk(patenttransfer, crosswalks, column = 'company'):    
    name_permno_dict = crosswalks.groupby('company')['permno'].apply(lambda g: g.values.tolist()).to_dict() 
    #exact match two companies
    #automatically duplicate permnos for each row using the merge; consume many memories; instead using the list
    patenttransfer['permno'] = patenttransfer[column].map(name_permno_dict)
    
    patenttransfer_aftermerge_notexactmatch = patenttransfer[patenttransfer['permno'].isna()]
    patenttransfer_aftermerge_exactmatch = patenttransfer[~patenttransfer['permno'].isna()]
    
    print("percent of exact match",
           patenttransfer_aftermerge_exactmatch.shape[0]/patenttransfer.shape[0])
    

    #fuzzy match two companies
    matches = match_strings(crosswalks['company'], patenttransfer_aftermerge_notexactmatch[column],
                            max_n_matches = 1,min_similarity = 0.5)
    matches = matches.sort_values(by = 'similarity', ascending = False)
    
    counts = matches['right_side'].apply(lambda x: x.split()[-1]).value_counts()
    endwords_ = list(counts[counts>= 100].index)
    counts = matches['left_side'].apply(lambda x: x.split()[-1]).value_counts()
    endwords2 = list(counts[counts>= 100].index)
    
    endwords_.extend(endwords2)
    
    patenttransfer_aftermerge_notexactmatch[column]= patenttransfer_aftermerge_notexactmatch[column].swifter.apply(remove_end_words, endwords = endwords_)
    patenttransfer_aftermerge_notexactmatch = patenttransfer_aftermerge_notexactmatch.drop_duplicates()

    
    crosswalks['company_remove_endwords'] = crosswalks['company'].swifter.apply(remove_end_words, endwords = endwords_)
    crosswalks.loc[crosswalks[crosswalks['company_remove_endwords'] == ''].index, 'company_remove_endwords'] = crosswalks[crosswalks['company_remove_endwords'] == '']['company']
    crosswalks.loc[crosswalks[crosswalks['company_remove_endwords'].isna()].index,'company_remove_endwords'] = crosswalks[crosswalks['company_remove_endwords'].isna()]['company']
    
    crosswalks_sub = crosswalks[['company_remove_endwords', 'permno']].drop_duplicates()
    name_permno_dict_2 = crosswalks_sub.groupby('company_remove_endwords')['permno'].apply(lambda g: g.values.tolist()).to_dict()     
    
    
    patenttransfer_aftermerge_notexactmatch['permno'] = patenttransfer_aftermerge_notexactmatch[column].map(name_permno_dict_2)
    

    patenttransfer_aftermerge_exactmatch = patenttransfer_aftermerge_exactmatch.append(patenttransfer_aftermerge_notexactmatch
        [(~patenttransfer_aftermerge_notexactmatch['permno'].isna())])
        
    patenttransfer_aftermerge_notexactmatch = patenttransfer_aftermerge_notexactmatch
    [(patenttransfer_aftermerge_notexactmatch['permno'].isna())]
    
    print("improved percent of exact match", patenttransfer_aftermerge_exactmatch.shape[0]/patenttransfer.shape[0])
    del patenttransfer

    
    #fuzzy match again based on refine company names
    matches = match_strings(crosswalks_sub['company_remove_endwords'], patenttransfer_aftermerge_notexactmatch[column], max_n_matches = 1,min_similarity = 0.5)
    
    matches_exact = matches[matches['similarity'] > 0.99999999]
    matches_notexact = matches[matches['similarity'] <=0.99999999]
    
    matches_notexact['overlap'] = matches_notexact.swifter.apply(find_overlap_words, axis = 1)
    matches_overlap = matches_notexact[matches_notexact['overlap']]
    matches_nonoverlap = matches_notexact[~matches_notexact['overlap']]
    
    matches_nonoverlap = matches_nonoverlap[~ matches_nonoverlap.swifter.apply(single_words_name, axis = 1)]
    #<0.8: keep if words have duplicate more than two; country name
    matches_final = pd.concat([matches_overlap,matches_nonoverlap[matches_nonoverlap['similarity']>= 0.8]])
    matches_final = pd.concat([matches_exact, matches_final])
    
    matches_final = matches_final.drop(columns = ['overlap'], axis = 1)
    matches_cut = matches_final.sort_values(by='similarity', ascending=False).drop_duplicates(subset = ['left_side', 'right_side'], keep = 'first')
    matches_cut = pd.merge(matches_cut, crosswalks_sub, left_on = 'left_side', right_on ='company_remove_endwords', how = 'left')
    
    matches_cut = matches_cut[['permno', 'right_side']].drop_duplicates()
    name_permno_dict_3 =matches_cut.groupby('right_side')['permno'].apply(lambda g: g.values.tolist()).to_dict()     
    patenttransfer_aftermerge_notexactmatch['permno'] = patenttransfer_aftermerge_notexactmatch[column].map(name_permno_dict_3)
    patenttransfer_aftermerge_exactmatch = patenttransfer_aftermerge_exactmatch.append(patenttransfer_aftermerge_notexactmatch)
    
    
    patenttransfer_aftermerge_exactmatch = patenttransfer_aftermerge_exactmatch.drop_duplicates([column])
    patenttransfer_aftermerge_exactmatch = patenttransfer_aftermerge_exactmatch.dropna()
    return patenttransfer_aftermerge_exactmatch

In [11]:
companynames['company'] = companynames['company'].astype(str)
crosswalks['company'] = crosswalks['company'].astype(str)

In [12]:
companynames=   build_companynames_permno_crosswalk(companynames, crosswalks)

percent of exact match 0.0038425066120243516


C:\Users\fjying\anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



improved percent of exact match 0.0038425066120243516


C:\Users\fjying\anaconda3\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
def find_not_unique_permno(x):
    if len(x) > 1:
        return True
    else:
        return False

In [22]:
companynames.to_csv("permno_processedallcompanyames_crosswalk.csv", index = False)